In [19]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna

In [20]:
df = pd.read_csv("data/final/final1.csv")
df = df[df['pot_doc'] != '[]']

y = np.array(df['documentid'])
X = np.array(df)

# df["pot_doc"].astype("O")
# df["pot_date"].astype("O")
# df["pot_price"].astype("O")
# df["pot_name"].astype("O")
# df["pot_address"].astype("O")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
x_column = list(df.columns)
x_column.remove('documentid')

X_train_df = pd.DataFrame(data = X_train, columns = df.columns)
X_test_df = pd.DataFrame(data = X_test, columns = df.columns)
y_train_df = pd.DataFrame(data = y_train, columns = ['documentid'])
y_test_df = pd.DataFrame(data = y_test, columns = ['documentid'])

X_train_df= X_train_df.drop(columns="documentid").drop(columns="Unnamed: 0")
X_test_df= X_test_df.drop(columns="documentid").drop(columns="Unnamed: 0")
display(X_test_df)

,pot_doc,pot_name,pot_price,pot_date,pot_address
0,"['00d0375035482', '00d0396141876', '00d0763327...","[11, 11, 19, 11]","[86, 75, 60, 75]","[0, 1, 0, 0]","[92, 88, 92, 88]"
1,"['00d0288011947', '00d0568013096', '00d0449498...","[16, 16, 16, 16, 16, 16, 16, 16, 13]","[67, 60, 67, 60, 100, 86, 57, 60, 57]","[0, 0, 0, 0, 0, 1, 0, 0, 0]","[90, 93, 93, 93, 93, 93, 93, 90, 93]"
2,"['00d0841246769', '00d0333208815', '00d0821795...","[3, 3, 3, 3, 3, 13, 16, 0, 3, 0, 0, 3, 3, 3, 0...","[100, 100, 100, 100, 75, 100, 67, 75, 50, 33, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[90, 93, 93, 90, 93, 93, 93, 93, 93, 93, 93, 9..."
3,['00d0721665358'],[19],[100],[0],[100]
4,"['00d0987973134', '00d0353042107', '00d0801912...","[14, 13, 13]","[50, 40, 67]","[0, 0, 1]","[83, 100, 100]"
...,...,...,...,...,...
94,"['00d0330533884', '00d0145695018', '00d0955195...","[17, 17, 17, 11, 17, 11, 17, 17, 11, 11, 17, 1...","[100, 100, 100, 100, 100, 50, 100, 100, 100, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[96, 88, 96, 96, 96, 96, 96, 92, 88, 92, 92, 8..."
95,"['00d0624439175', '00d0726523245', '00d0447521...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 14, 2, 2, 13...","[75, 86, 67, 100, 100, 100, 67, 100, 100, 67, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[97, 93, 97, 97, 93, 97, 97, 97, 97, 97, 97, 9..."
96,['00d0685723809'],[12],[100],[1],[93]
97,"['00d0403711886', '00d0546591203', '00d0989196...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, ...","[44, 100, 100, 57, 100, 100, 50, 57, 57, 67, 5...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[95, 95, 95, 95, 95, 91, 95, 91, 95, 91, 95, 9..."


In [21]:
def compute_accuracy(x, y, z,a):
    results = []
    for row in X_train_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*(100-eval(row[1]['pot_price'])[idx])/100)**2 
            + (y*(1-eval(row[1]['pot_date'])[idx]))**2
            + (z*(100-eval(row[1]['pot_address'])[idx])/100)**2
            #+ (a*eval(row[1]['pot_name'])[idx])**2
            )
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return np.sum(results == np.array(y_train_df['documentid']))/len(X_train_df)
    
def compute_accuracy2(x, y, z, a):
    results = []
    for row in X_train_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*(100-eval(row[1]['pot_price'])[idx])/100)**2 
            + (y*(1-eval(row[1]['pot_date'])[idx]))**2
            + (z*(100-eval(row[1]['pot_address'])[idx])/100)**2
            #+ (a*eval(row[1]['pot_name'])[idx])**2
            )
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return results

In [22]:

# dfs = pd.DataFrame()
# ss = pd.DataFrame()

# dfs['result'] = ss.apply(lambda row: (row['documentid'] in row['pot_doc']), axis=1)
# print(dfs)
# num_trues = dfs['result'].value_counts()[True]
# print(num_trues)
def objective(trial):
    x = trial.suggest_float('x', 0, 10)
    y = trial.suggest_float('y', 0, 10)
    #z = trial.suggest_float('z', 0, 1)
    #a = trial.suggest_float('a', 0, 10)
    return 1-compute_accuracy(x,y,1,0)

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params  # E.g. {'x': 2.002108042}

[I 2023-01-29 05:27:23,509] A new study created in memory with name: no-name-dc6b2331-c609-4192-a8a0-eed4b70dc347
[I 2023-01-29 05:27:24,128] Trial 0 finished with value: 0.1544303797468355 and parameters: {'x': 1.528011090959478, 'y': 2.04815658732905}. Best is trial 0 with value: 0.1544303797468355.
[I 2023-01-29 05:27:24,739] Trial 1 finished with value: 0.1544303797468355 and parameters: {'x': 5.921037286037617, 'y': 9.602611436921835}. Best is trial 0 with value: 0.1544303797468355.
[I 2023-01-29 05:27:25,337] Trial 2 finished with value: 0.1544303797468355 and parameters: {'x': 4.525081209092786, 'y': 6.4547810520911595}. Best is trial 0 with value: 0.1544303797468355.
[I 2023-01-29 05:27:25,937] Trial 3 finished with value: 0.1544303797468355 and parameters: {'x': 4.294379948070633, 'y': 4.9303801716757025}. Best is trial 0 with value: 0.1544303797468355.
[I 2023-01-29 05:27:26,537] Trial 4 finished with value: 0.2126582278481013 and parameters: {'x': 9.523782604407888, 'y': 1.8

{'x': 1.1655517138264777, 'y': 5.579638633922475}

In [23]:
res = compute_accuracy2(study.best_params['x'], 1, study.best_params['z'], 0)
real = list(y_train_df['documentid'])
print(len(res), len(real))

KeyError: 'z'

In [ ]:
c = 0
for i in range(len(res)):
    if res[i] != real[i] and real[i] in eval(X_train_df['pot_doc'][i]):
        print(res[i], real[i])
        c += 1
print(c)

00d0201738305 00d0568013096
00d0954032591 00d0628164309
00d0511840834 00d0734092781
00d0358154075 00d0982931302
00d0106074517 00d0967073777
00d0648982621 00d0627668053
00d0396141876 00d0375035482
00d0546591203 00d0734493835
00d0975647950 00d0530467663
00d0167381962 00d0977582300
00d0989196649 00d0630325043
00d0250566763 00d0619690945
00d0954032591 00d0452553939
00d0954032591 00d0880901672
00d0954032591 00d0970540567
00d0954032591 00d0639354292
00d0724139826 00d0242626168
00d0542653551 00d0944630849
00d0542888628 00d0485748096
00d0390492165 00d0259552507
00d0390492165 00d0717257644
00d0250566763 00d0997573916
00d0954032591 00d0558270937
00d0243046961 00d0542157895
00d0674212079 00d0742987288
00d0645069761 00d0417300467
00d0763327632 00d0432757654
00d0954032591 00d0408489357
00d0212522180 00d0177909385
00d0954032591 00d0602427476
00d0992127574 00d0394257241
00d0382500008 00d0805870344
00d0375035482 00d0763327632
00d0202365816 00d0551541808
00d0627357411 00d0500564944
00d0511840834 00d018